# Día 2: Métodos Mágicos (Magic Methods)

## Descripción General

Los métodos mágicos (también conocidos como *dunder methods* por "double underscore") son métodos especiales en Python que comienzan y terminan con doble guion bajo (`__`). Estos métodos permiten que tus clases personalizadas se integren perfectamente con las operaciones integradas de Python, como la impresión, comparación, iteración y operaciones aritméticas.

En este notebook, exploraremos los métodos mágicos más comunes y útiles que te permitirán crear clases que se comportan de manera natural y pythónica.

## Objetivos de Aprendizaje

Al finalizar este notebook, serás capaz de:

1. Comprender qué son los métodos mágicos y por qué son importantes en Python
2. Implementar `__str__` y `__repr__` para controlar la representación de objetos
3. Usar `__eq__`, `__lt__`, y otros métodos de comparación para definir igualdad y orden
4. Implementar `__len__` y `__getitem__` para crear objetos tipo contenedor
5. Utilizar `__iter__` y `__next__` para hacer objetos iterables

## 1. Introducción a los Métodos Mágicos

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Estás construyendo una clase `Dataset` para tu pipeline de ML. Sin métodos mágicos:
- No puedes hacer `len(dataset)` → tienes que llamar `dataset.get_length()`
- No puedes hacer `dataset[0]` → tienes que llamar `dataset.get_item(0)`
- No puedes hacer `for item in dataset` → no es iterable
- `print(dataset)` muestra `<__main__.Dataset object at 0x...>` → inútil

Resultado: Tu clase no se integra con Python, código verboso y poco intuitivo.

**Ejemplo concreto para juniors**:

Tienes clase `Point(x, y)` para geometría. Sin métodos mágicos:
```python
p1 = Point(3, 4)
print(p1)  # <__main__.Point object at 0x...> ❌
p1 == Point(3, 4)  # False (diferentes objetos) ❌
p1 + p2  # TypeError ❌
```

Con métodos mágicos:
```python
p1 = Point(3, 4)
print(p1)  # Point(3, 4) ✅
p1 == Point(3, 4)  # True ✅
p1 + p2  # Point(5, 7) ✅
```

**Consecuencias de NO usarlos**:
- **Clases no pythónicas** → no se comportan como tipos built-in
- **Código verboso** → `obj.get_item(0)` en lugar de `obj[0]`
- **No componible** → no funciona con funciones built-in (len, iter, etc.)
- **Difícil de debuggear** → print muestra basura en lugar de info útil

### 📚 El Concepto

**Definición técnica**:

Métodos mágicos (dunder methods) son métodos especiales con nombres `__nombre__` que Python llama automáticamente en respuesta a operaciones específicas. Son la interfaz entre tus clases y el modelo de datos de Python.

**Cómo funciona internamente**:
1. Escribes operación: `len(obj)`, `obj[0]`, `obj + other`
2. Python busca método mágico correspondiente: `__len__`, `__getitem__`, `__add__`
3. Si existe → llama al método
4. Si no existe → lanza TypeError

**Terminología clave**:
- **Dunder**: Double UNDERscore (doble guion bajo)
- **Magic methods**: Métodos que Python llama automáticamente
- **Data model**: Protocolo que define cómo objetos se comportan
- **Protocol**: Conjunto de métodos que definen un comportamiento

**Categorías principales**:
1. **Representación**: `__str__`, `__repr__`
2. **Comparación**: `__eq__`, `__lt__`, `__le__`, `__gt__`, `__ge__`
3. **Contenedores**: `__len__`, `__getitem__`, `__setitem__`, `__contains__`
4. **Iteración**: `__iter__`, `__next__`
5. **Aritméticos**: `__add__`, `__sub__`, `__mul__`, `__truediv__`
6. **Context managers**: `__enter__`, `__exit__`

In [ ]:
# BAD: Class without magic methods
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

p1 = Point(3, 4)
print(p1)  # Output: <__main__.Point object at 0x...> - Not helpful!
print(p1 == Point(3, 4))  # False - Different objects, even with same values

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. Métodos mágicos son la **interfaz** entre tus clases y Python
2. Python los llama **automáticamente** → no los llamas directamente
3. Hacen tus clases **pythónicas** → se comportan como tipos built-in
4. **No inventes** tus propios dunder methods → solo usa los oficiales

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Quiero que mi clase se comporte como [tipo built-in]?"
  - Como lista → implementa `__len__`, `__getitem__`, `__iter__`
  - Como número → implementa `__add__`, `__sub__`, `__mul__`
  - Como dict → implementa `__getitem__`, `__setitem__`, `__contains__`

- **Pregúntate**: "¿Qué operaciones naturales tiene mi clase?"
  - Si tiene "tamaño" → `__len__`
  - Si tiene "elementos" → `__getitem__`, `__iter__`
  - Si se puede "comparar" → `__eq__`, `__lt__`

**Cuándo usar / NO usar**:
- ✅ **Usar cuando**:
  - Quieres que tu clase sea pythónica
  - Necesitas integración con built-ins (len, iter, etc.)
  - Quieres operadores naturales (+, -, ==, <)
  - Debugging (print debe mostrar info útil)
  
- ❌ **NO usar cuando**:
  - No hay operación natural (forzar `+` donde no tiene sentido)
  - Comportamiento no obvio (sorprende al usuario)
  - Solo por "lucirse" → simplicidad > magia

**Referencia oficial:** [Data Model - Python Documentation](https://docs.python.org/3/reference/datamodel.html)

## 2. __str__ y __repr__: Representación de Objetos

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Estás debuggeando un pipeline de ML con 50 objetos `Model`, `Dataset`, `Preprocessor`. Cuando imprimes un objeto para debuggear:
```python
print(model)  # <__main__.Model object at 0x7f8b3c4d5e80>
```

¿Qué modelo es? ¿Qué parámetros tiene? ¿Está entrenado? **No tienes idea** 💥

Con `__repr__` bien implementado:
```python
print(model)  # Model(type='RandomForest', n_estimators=100, trained=True)
```

Ahora sabes exactamente qué es y su estado ✅

**Ejemplo concreto para juniors**:

Tienes clase `User(name, email, age)`. Sin `__str__`/`__repr__`:
- Logs muestran `<User object at 0x...>` → inútil para debugging
- No puedes ver qué usuario causó el error
- Pierdes 30 minutos buscando el problema

Con `__str__`/`__repr__`:
- Logs muestran `User('Alice', 'alice@example.com', 25)`
- Identificas el problema inmediatamente

**Consecuencias de NO usarlos**:
- **Debugging difícil** → no sabes qué objeto estás viendo
- **Logs inútiles** → solo ves direcciones de memoria
- **Mala experiencia de desarrollo** → pierdes tiempo investigando
- **No profesional** → código parece amateur

### 📚 El Concepto

**Definición técnica**:

- `__str__`: Representación "amigable" para usuarios finales (legible, informal)
- `__repr__`: Representación "técnica" para desarrolladores (sin ambigüedad, idealmente evaluable)

**Cómo funciona internamente**:
1. `print(obj)` → llama `str(obj)` → busca `__str__`
2. Si no existe `__str__` → usa `__repr__` como fallback
3. `repr(obj)` → llama `__repr__`
4. REPL/debugger → usa `__repr__`
5. Contenedores (list, dict) → usan `__repr__` de elementos

**Terminología clave**:
- **str()**: Conversión a string "amigable"
- **repr()**: Representación "oficial" del objeto
- **Evaluable**: `eval(repr(obj)) == obj` (ideal pero no siempre posible)
- **Unambiguous**: Sin ambigüedad sobre qué es el objeto

**Regla de oro**:
```python
__str__: Para humanos ("Point at (3, 4)")
__repr__: Para desarrolladores ("Point(3, 4)")
```

In [ ]:
# GOOD: Class with __str__ and __repr__
class Point:
    """
    Represent a point in 2D space.
    
    :param x: X coordinate
    :type x: float
    :param y: Y coordinate
    :type y: float
    """
    def __init__(self, x: float, y: float) -> None:
        self.x = x
        self.y = y
    
    def __str__(self) -> str:
        """Return user-friendly string representation."""
        return f"Point at ({self.x}, {self.y})"
    
    def __repr__(self) -> str:
        """Return developer-friendly string representation."""
        return f"Point({self.x}, {self.y})"

p1 = Point(3, 4)
print(p1)  # Uses __str__: Point at (3, 4)
print(repr(p1))  # Uses __repr__: Point(3, 4)
print([p1])  # Lists use __repr__: [Point(3, 4)]

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. `__str__` es para **usuarios** (legible), `__repr__` es para **desarrolladores** (sin ambigüedad)
2. Si solo implementas uno, **implementa `__repr__`** (Python lo usa como fallback)
3. `__repr__` debe ser **evaluable** cuando sea posible: `eval(repr(obj)) == obj`
4. Contenedores (list, dict) **siempre usan `__repr__`** de sus elementos

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Qué info necesito para entender este objeto al debuggear?"
  - Tipo del objeto
  - Valores de atributos clave
  - Estado actual (si aplica)

- **Pregúntate**: "¿Puedo recrear el objeto con esta representación?"
  - SÍ → excelente `__repr__`
  - NO pero es claro → aceptable

**Cuándo usar / NO usar**:
- ✅ **SIEMPRE implementa `__repr__`**:
  - Toda clase debe tener `__repr__`
  - Formato: `ClassName(arg1, arg2, ...)`
  - Incluye valores de atributos importantes
  
- ✅ **Implementa `__str__` cuando**:
  - Necesitas formato más amigable para usuarios
  - Representación larga/compleja en `__repr__`
  - Output para end-users (no desarrolladores)
  
- ❌ **NO hagas**:
  - `__repr__` que no muestra info útil
  - `__str__` y `__repr__` idénticos (solo implementa `__repr__`)
  - Representaciones ambiguas

**Ejemplos de buenas representaciones**:
```python
# ✅ BIEN: Evaluable y claro
Point(3, 4)
datetime.datetime(2024, 1, 15, 10, 30, 0)
User('alice', 'alice@example.com', age=25)

# ✅ BIEN: No evaluable pero muy claro
Model(type='RandomForest', trained=True, accuracy=0.95)
<User: alice (alice@example.com)>

# ❌ MAL: No dice nada útil
<__main__.Point object at 0x7f8b3c4d5e80>
<Point instance>
```

**Diferencia en uso**:
```python
user = User('Alice', 25)

print(user)        # Usa __str__: "Alice (25 years old)"
repr(user)         # Usa __repr__: "User('Alice', 25)"
str(user)          # Usa __str__: "Alice (25 years old)"
[user]             # Usa __repr__: [User('Alice', 25)]
f"{user}"          # Usa __str__: "Alice (25 years old)"
f"{user!r}"        # Usa __repr__: "User('Alice', 25)"
```

**Referencia oficial:** [object.__str__ and object.__repr__](https://docs.python.org/3/reference/datamodel.html#object.__repr__)

### Pregunta de Comprensión

¿Cuál es la diferencia entre `__str__` y `__repr__`, y cuándo se usa cada uno?

## 3. Métodos de Comparación: __eq__, __lt__, __le__, __gt__, __ge__

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Estás construyendo un sistema de priorización de tareas de ML. Tienes clase `Task(priority, deadline, cost)`. Sin métodos de comparación:
```python
task1 = Task(priority=5, deadline='2024-01-15', cost=100)
task2 = Task(priority=5, deadline='2024-01-15', cost=100)

task1 == task2  # False ❌ (diferentes objetos en memoria)
sorted(tasks)   # TypeError ❌ (no se puede ordenar)
max(tasks)      # TypeError ❌ (no se puede comparar)
```

Con métodos de comparación:
```python
task1 == task2  # True ✅ (mismos valores)
sorted(tasks)   # [task3, task1, task2] ✅ (ordenado por prioridad)
max(tasks)      # task1 ✅ (mayor prioridad)
```

**Ejemplo concreto para juniors**:

Tienes clase `Product(name, price, rating)`. Necesitas:
- Comparar si dos productos son iguales (mismo nombre y precio)
- Ordenar productos por precio
- Encontrar el producto más caro
- Filtrar productos con rating > 4.0

Sin comparación: Tienes que escribir funciones custom para cada operación.
Con comparación: Usas `==`, `<`, `sorted()`, `max()`, `filter()` directamente.

**Consecuencias de NO usarlos**:
- **No puedes usar `sorted()`** → tienes que implementar ordenamiento manual
- **No puedes usar `max()`/`min()`** → loops manuales
- **Comparación por identidad** → `obj1 == obj2` siempre False aunque tengan mismos valores
- **No funciona con algoritmos** → muchos algoritmos requieren comparación

### 📚 El Concepto

**Definición técnica**:

Métodos de comparación definen cómo objetos de tu clase se comparan entre sí:
- `__eq__(self, other)`: Igualdad (`==`)
- `__ne__(self, other)`: Desigualdad (`!=`) - Python lo deriva de `__eq__`
- `__lt__(self, other)`: Menor que (`<`)
- `__le__(self, other)`: Menor o igual (`<=`)
- `__gt__(self, other)`: Mayor que (`>`)
- `__ge__(self, other)`: Mayor o igual (`>=`)

**Cómo funciona internamente**:
1. Escribes: `obj1 == obj2`
2. Python llama: `obj1.__eq__(obj2)`
3. Si no existe `__eq__` → compara por identidad (`id(obj1) == id(obj2)`)
4. Si retorna `NotImplemented` → Python intenta `obj2.__eq__(obj1)`

**Terminología clave**:
- **Rich comparison**: Métodos de comparación completos
- **NotImplemented**: Valor especial (NO excepción) que indica "no sé comparar con este tipo"
- **total_ordering**: Decorador que genera métodos faltantes desde `__eq__` y uno de `__lt__`/`__le__`/`__gt__`/`__ge__`
- **Reflexive**: `a == a` debe ser True
- **Symmetric**: Si `a == b` entonces `b == a`
- **Transitive**: Si `a == b` y `b == c` entonces `a == c`

**Patrón recomendado**:
```python
from functools import total_ordering

@total_ordering
class MyClass:
    def __eq__(self, other):
        if not isinstance(other, MyClass):
            return NotImplemented
        return self.value == other.value
    
    def __lt__(self, other):
        if not isinstance(other, MyClass):
            return NotImplemented
        return self.value < other.value
```

In [ ]:
# BAD: Without comparison methods
class Person:
    def __init__(self, name: str, age: int) -> None:
        self.name = name
        self.age = age

p1 = Person("Alice", 30)
p2 = Person("Alice", 30)
print(p1 == p2)  # False - Different objects!
# print(p1 < p2)  # TypeError: '<' not supported

In [ ]:
# GOOD: With comparison methods
class Person:
    """
    Represent a person with name and age.
    
    :param name: Person's name
    :type name: str
    :param age: Person's age
    :type age: int
    """
    def __init__(self, name: str, age: int) -> None:
        self.name = name
        self.age = age
    
    def __eq__(self, other: object) -> bool:
        """Check equality based on name and age."""
        if not isinstance(other, Person):
            return NotImplemented
        return self.name == other.name and self.age == other.age
    
    def __lt__(self, other: 'Person') -> bool:
        """Compare by age for sorting."""
        if not isinstance(other, Person):
            return NotImplemented
        return self.age < other.age
    
    def __repr__(self) -> str:
        return f"Person('{self.name}', {self.age})"

p1 = Person("Alice", 30)
p2 = Person("Alice", 30)
p3 = Person("Bob", 25)

print(p1 == p2)  # True - Same values!
print(p1 < p3)  # False - Alice is older
print(p3 < p1)  # True - Bob is younger

# Sorting works now!
people = [p1, p3, Person("Charlie", 35)]
print(sorted(people))  # Sorted by age

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. **SIEMPRE retorna `NotImplemented`** para tipos incompatibles (NO `False` o excepción)
2. Usa **`@total_ordering`** para generar métodos faltantes (solo necesitas `__eq__` + uno de `__lt__`)
3. `__eq__` debe ser **reflexivo, simétrico y transitivo**
4. Si implementas `__eq__`, también implementa **`__hash__`** (o hazlo unhashable con `__hash__ = None`)

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Tiene sentido comparar objetos de esta clase?"
  - SÍ (productos por precio, tareas por prioridad) → implementa comparación
  - NO (conexiones DB, archivos abiertos) → no implementes

- **Pregúntate**: "¿Por qué criterio se comparan?"
  - Un solo criterio claro (precio, edad, prioridad) → implementa
  - Múltiples criterios ambiguos → considera no implementar o usar key functions

**Cuándo usar / NO usar**:
- ✅ **Usar cuando**:
  - Objetos tienen orden natural (números, fechas, prioridades)
  - Necesitas `sorted()`, `max()`, `min()`
  - Comparación por valores (no identidad)
  - Objetos en estructuras ordenadas (heaps, sorted lists)
  
- ❌ **NO usar cuando**:
  - No hay orden natural obvio
  - Múltiples formas válidas de ordenar → usa `key=` en sorted
  - Comparación no tiene sentido semántico

**Patrón con total_ordering**:
```python
from functools import total_ordering

@total_ordering  # Genera __le__, __gt__, __ge__ automáticamente
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def __eq__(self, other):
        if not isinstance(other, Person):
            return NotImplemented
        return self.age == other.age
    
    def __lt__(self, other):  # Solo necesitas este + __eq__
        if not isinstance(other, Person):
            return NotImplemented
        return self.age < other.age
```

**Por qué NotImplemented (no False)**:
```python
# ❌ MAL: Retornar False
def __eq__(self, other):
    if not isinstance(other, MyClass):
        return False  # Dice que son diferentes
    return self.value == other.value

# Problema:
my_obj == "string"  # False (parece que son comparables)
"string" == my_obj  # False (pero Python no puede intentar al revés)

# ✅ BIEN: Retornar NotImplemented
def __eq__(self, other):
    if not isinstance(other, MyClass):
        return NotImplemented  # "No sé comparar con esto"
    return self.value == other.value

# Beneficio:
my_obj == "string"  # Python intenta "string".__eq__(my_obj)
# Si ambos retornan NotImplemented → False (correcto)
```

**Relación con __hash__**:
```python
# Si implementas __eq__, debes considerar __hash__

# Opción 1: Hacer unhashable (no puede estar en set/dict)
class MutablePoint:
    __hash__ = None  # Explícitamente unhashable
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

# Opción 2: Implementar __hash__ (solo si objeto es inmutable)
class ImmutablePoint:
    def __init__(self, x, y):
        self._x = x  # Inmutable
        self._y = y
    
    def __eq__(self, other):
        return self._x == other._x and self._y == other._y
    
    def __hash__(self):
        return hash((self._x, self._y))
```

**Referencia oficial:** [Rich Comparison Methods](https://docs.python.org/3/reference/datamodel.html#object.__lt__)

In [ ]:
# Using @total_ordering to reduce boilerplate
from functools import total_ordering

@total_ordering
class Product:
    """
    Represent a product with name and price.
    
    :param name: Product name
    :type name: str
    :param price: Product price
    :type price: float
    """
    def __init__(self, name: str, price: float) -> None:
        self.name = name
        self.price = price
    
    def __eq__(self, other: object) -> bool:
        if not isinstance(other, Product):
            return NotImplemented
        return self.price == other.price
    
    def __lt__(self, other: 'Product') -> bool:
        if not isinstance(other, Product):
            return NotImplemented
        return self.price < other.price
    
    def __repr__(self) -> str:
        return f"Product('{self.name}', ${self.price})"

# Now all comparison operators work!
apple = Product("Apple", 0.5)
banana = Product("Banana", 0.3)
orange = Product("Orange", 0.8)

print(apple > banana)  # True
print(apple <= orange)  # True
print(banana >= orange)  # False

## 4. Métodos de Contenedor: __len__ y __getitem__

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Estás construyendo una clase `DataBatch` para tu pipeline de ML que carga datos en lotes. Sin `__len__` y `__getitem__`:
```python
batch = DataBatch(data)

len(batch)           # TypeError ❌
batch[0]             # TypeError ❌
for item in batch:   # TypeError ❌ (no iterable)
batch[:10]           # TypeError ❌ (no slicing)
```

Con `__len__` y `__getitem__`:
```python
len(batch)           # 1000 ✅
batch[0]             # first_item ✅
for item in batch:   # ✅ (funciona automáticamente)
batch[:10]           # first_10_items ✅
```

**Ejemplo concreto para juniors**:

Tienes clase `Playlist` que almacena canciones. Quieres:
- `len(playlist)` → número de canciones
- `playlist[0]` → primera canción
- `playlist[-1]` → última canción
- `for song in playlist` → iterar canciones
- `playlist[2:5]` → canciones 2, 3, 4

Sin estos métodos: Tienes que llamar `playlist.get_song(0)`, `playlist.get_count()`, etc.
Con estos métodos: Tu clase se comporta como una lista nativa.

**Consecuencias de NO usarlos**:
- **No puedes usar `len()`** → tienes que llamar método custom
- **No puedes usar `[]`** → sintaxis verbosa
- **No es iterable** → no funciona en loops `for`
- **No funciona con algoritmos** → muchos esperan protocolo de secuencia

### 📚 El Concepto

**Definición técnica**:

- `__len__(self)`: Retorna longitud del contenedor (debe ser int >= 0)
- `__getitem__(self, key)`: Retorna elemento en posición `key` (soporta int, slice, etc.)

**Cómo funciona internamente**:
1. `len(obj)` → llama `obj.__len__()`
2. `obj[key]` → llama `obj.__getitem__(key)`
3. `for item in obj` → Python llama `obj.__getitem__(0)`, `obj.__getitem__(1)`, ... hasta `IndexError`
4. `obj[start:stop:step]` → Python crea objeto `slice(start, stop, step)` y llama `obj.__getitem__(slice_obj)`

**Terminología clave**:
- **Container protocol**: Implementar `__len__` y `__getitem__`
- **Sequence protocol**: Container + orden + indexación por enteros
- **Slicing**: Acceso a sub-secuencias con `[start:stop:step]`
- **Negative indexing**: `obj[-1]` accede desde el final

**Bonus**: Si implementas `__getitem__`, tu objeto es **automáticamente iterable** (Python itera llamando `[0]`, `[1]`, ... hasta `IndexError`)

In [ ]:
# GOOD: Custom container class
class Playlist:
    """
    Represent a music playlist.
    
    :param name: Playlist name
    :type name: str
    """
    def __init__(self, name: str) -> None:
        self.name = name
        self._songs = []
    
    def add_song(self, song: str) -> None:
        """Add a song to the playlist."""
        self._songs.append(song)
    
    def __len__(self) -> int:
        """Return number of songs in playlist."""
        return len(self._songs)
    
    def __getitem__(self, index: int) -> str:
        """Get song at index."""
        return self._songs[index]
    
    def __repr__(self) -> str:
        return f"Playlist('{self.name}', {len(self)} songs)"

playlist = Playlist("My Favorites")
playlist.add_song("Song A")
playlist.add_song("Song B")
playlist.add_song("Song C")

print(len(playlist))  # 3 - __len__ works!
print(playlist[0])  # "Song A" - __getitem__ works!
print(playlist[-1])  # "Song C" - Negative indexing works!

# Can use in for loop because __getitem__ is defined
for song in playlist:
    print(f"Playing: {song}")

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. `__len__` debe retornar **int >= 0** (no float, no None)
2. `__getitem__` hace tu objeto **automáticamente iterable**
3. `__getitem__` debe **lanzar `IndexError`** cuando índice fuera de rango (no retornar None)
4. Para slicing, `key` es objeto **`slice`** (no tupla de ints)

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Mi clase contiene una colección de elementos?"
  - SÍ (playlist, dataset, batch) → implementa `__len__` y `__getitem__`
  - NO (conexión, configuración) → no implementes

- **Pregúntate**: "¿Tiene sentido acceder por índice?"
  - SÍ (orden importa) → implementa `__getitem__`
  - NO (sin orden, como set) → solo `__len__` y `__contains__`

**Cuándo usar / NO usar**:
- ✅ **Usar cuando**:
  - Tu clase envuelve una colección (lista, array, etc.)
  - Elementos tienen orden natural
  - Quieres sintaxis `obj[i]` en lugar de `obj.get(i)`
  - Necesitas que sea iterable
  
- ❌ **NO usar cuando**:
  - No hay colección interna
  - Acceso por índice no tiene sentido
  - Orden no importa (considera `__contains__` en su lugar)

**Manejo de slicing**:
```python
def __getitem__(self, key):
    # key puede ser int o slice
    if isinstance(key, slice):
        # Manejar slicing: obj[start:stop:step]
        start, stop, step = key.indices(len(self))
        return [self._items[i] for i in range(start, stop, step)]
    elif isinstance(key, int):
        # Manejar índice simple: obj[5]
        if key < 0:
            key += len(self)  # Convertir índice negativo
        if key < 0 or key >= len(self):
            raise IndexError("Index out of range")
        return self._items[key]
    else:
        raise TypeError(f"Indices must be integers or slices, not {type(key).__name__}")
```

**Iteración automática**:
```python
# Solo con __getitem__, tu clase es iterable
class MyList:
    def __init__(self, items):
        self._items = items
    
    def __getitem__(self, index):
        return self._items[index]

# Funciona automáticamente:
my_list = MyList([1, 2, 3])
for item in my_list:  # Python llama __getitem__(0), __getitem__(1), ...
    print(item)
```

**Contenedor completo (mutable)**:
```python
class MutableContainer:
    def __len__(self):
        return len(self._items)
    
    def __getitem__(self, key):
        return self._items[key]
    
    def __setitem__(self, key, value):  # Para obj[key] = value
        self._items[key] = value
    
    def __delitem__(self, key):  # Para del obj[key]
        del self._items[key]
    
    def __contains__(self, item):  # Para item in obj
        return item in self._items
```

**Herencia de collections.abc**:
```python
from collections.abc import Sequence

class MySequence(Sequence):
    # Solo necesitas implementar __len__ y __getitem__
    # Sequence proporciona: __contains__, __iter__, __reversed__, index, count
    
    def __len__(self):
        return len(self._items)
    
    def __getitem__(self, index):
        return self._items[index]
```

**Referencia oficial:** [Emulating Container Types](https://docs.python.org/3/reference/datamodel.html#emulating-container-types)

### Pregunta de Comprensión

¿Qué métodos mágicos necesitas implementar para que tu clase se comporte como una lista?

## 5. Métodos de Iteración: __iter__ y __next__

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Estás construyendo un `DataLoader` que carga batches de datos bajo demanda desde disco (no cabe todo en RAM). Sin `__iter__`/`__next__`:
```python
loader = DataLoader('huge_dataset.csv')

for batch in loader:  # TypeError ❌ (no iterable)
    train_model(batch)

# Tienes que hacer:
while loader.has_next():  # Feo y verboso
    batch = loader.get_next()
    train_model(batch)
```

Con `__iter__`/`__next__`:
```python
for batch in loader:  # ✅ Pythonic y limpio
    train_model(batch)
```

**Ejemplo concreto para juniors**:

Tienes clase `Countdown(start)` que cuenta regresivamente. Quieres:
```python
for num in Countdown(5):
    print(num)  # 5, 4, 3, 2, 1, 0
```

Sin `__iter__`/`__next__`: No puedes usar `for`, tienes que hacer loops manuales.
Con `__iter__`/`__next__`: Tu clase funciona naturalmente en loops.

**Consecuencias de NO usarlos**:
- **No funciona en loops `for`** → sintaxis verbosa
- **No funciona con comprehensions** → `[x for x in obj]` falla
- **No funciona con `next()`** → no puedes consumir elemento por elemento
- **No es lazy** → si usas `__getitem__`, Python asume secuencia finita

### 📚 El Concepto

**Definición técnica**:

- `__iter__(self)`: Retorna objeto iterador (usualmente `self`)
- `__next__(self)`: Retorna siguiente elemento o lanza `StopIteration` cuando termina

**Cómo funciona internamente**:
1. `for item in obj` → Python llama `iter(obj)` → llama `obj.__iter__()`
2. `__iter__()` retorna iterador (objeto con `__next__`)
3. Python llama `next(iterator)` → llama `iterator.__next__()`
4. `__next__()` retorna siguiente valor
5. Cuando no hay más elementos → `__next__()` lanza `StopIteration`
6. Python captura `StopIteration` y termina el loop

**Terminología clave**:
- **Iterable**: Objeto con `__iter__` que retorna iterador
- **Iterator**: Objeto con `__next__` que produce valores
- **StopIteration**: Excepción que señala fin de iteración
- **Iterator protocol**: Implementar `__iter__` y `__next__`

**Diferencia Iterable vs Iterator**:
```python
# Iterable: puede ser iterado (tiene __iter__)
my_list = [1, 2, 3]  # Iterable

# Iterator: produce valores (tiene __next__)
my_iter = iter(my_list)  # Iterator
next(my_iter)  # 1
next(my_iter)  # 2
```

**Patrón común**: Clase es iterable E iterador (retorna `self` en `__iter__`)

In [ ]:
# GOOD: Custom iterator
class Countdown:
    """
    Iterator that counts down from a starting number.
    
    :param start: Starting number
    :type start: int
    """
    def __init__(self, start: int) -> None:
        self.start = start
        self.current = start
    
    def __iter__(self) -> 'Countdown':
        """Return the iterator object (self)."""
        self.current = self.start  # Reset for reuse
        return self
    
    def __next__(self) -> int:
        """Return the next value in the countdown."""
        if self.current < 0:
            raise StopIteration
        value = self.current
        self.current -= 1
        return value

countdown = Countdown(5)
for num in countdown:
    print(num)  # 5, 4, 3, 2, 1, 0

# Can iterate again
print(list(countdown))  # [5, 4, 3, 2, 1, 0]

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. `__iter__` debe retornar **objeto con `__next__`** (usualmente `self`)
2. `__next__` debe **lanzar `StopIteration`** cuando termina (no retornar None)
3. Iteradores se **agotan** (solo 1 iteración) → para reusar, `__iter__` debe resetear estado
4. Si implementas `__getitem__`, ya eres iterable (Python itera con índices)

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Necesito generar valores bajo demanda?"
  - SÍ (archivos grandes, streams, secuencias infinitas) → `__iter__`/`__next__`
  - NO (colección finita en memoria) → `__getitem__` es suficiente

- **Pregúntate**: "¿La secuencia es infinita o muy grande?"
  - SÍ → DEBES usar `__iter__`/`__next__` (lazy)
  - NO → `__getitem__` puede ser más simple

**Cuándo usar / NO usar**:
- ✅ **Usar cuando**:
  - Generas valores bajo demanda (lazy evaluation)
  - Secuencias infinitas (Fibonacci, números primos)
  - Datos no caben en memoria (archivos, streams)
  - Necesitas mantener estado entre iteraciones
  - Iteración compleja (no solo índices secuenciales)
  
- ❌ **NO usar cuando**:
  - Colección finita en memoria → `__getitem__` más simple
  - Necesitas acceso aleatorio → `__getitem__` mejor
  - Necesitas `len()` → iteradores no tienen longitud

**Patrón 1: Clase es iterable E iterador**:
```python
class Countdown:
    def __init__(self, start):
        self.start = start
        self.current = start
    
    def __iter__(self):
        self.current = self.start  # Reset para reusar
        return self  # Retorna self (es su propio iterador)
    
    def __next__(self):
        if self.current < 0:
            raise StopIteration
        value = self.current
        self.current -= 1
        return value
```

**Patrón 2: Iterable retorna iterador separado**:
```python
class MyRange:
    def __init__(self, start, stop):
        self.start = start
        self.stop = stop
    
    def __iter__(self):
        return MyRangeIterator(self.start, self.stop)  # Nuevo iterador

class MyRangeIterator:
    def __init__(self, start, stop):
        self.current = start
        self.stop = stop
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.current >= self.stop:
            raise StopIteration
        value = self.current
        self.current += 1
        return value
```

**Ventaja del Patrón 2**: Múltiples iteraciones simultáneas
```python
my_range = MyRange(0, 5)

# Dos iteraciones independientes
iter1 = iter(my_range)
iter2 = iter(my_range)

next(iter1)  # 0
next(iter2)  # 0
next(iter1)  # 1
next(iter2)  # 1
```

**Iterador infinito**:
```python
class InfiniteCounter:
    def __init__(self, start=0):
        self.current = start
    
    def __iter__(self):
        return self
    
    def __next__(self):
        value = self.current
        self.current += 1
        return value  # Nunca lanza StopIteration

# Uso con límite
counter = InfiniteCounter()
for i, num in enumerate(counter):
    if i >= 10:
        break
    print(num)
```

**Comparación con generator**:
```python
# Con __iter__/__next__ (más control)
class Countdown:
    def __init__(self, start):
        self.start = start
    
    def __iter__(self):
        current = self.start
        while current >= 0:
            yield current
            current -= 1

# Con generator function (más simple)
def countdown(start):
    while start >= 0:
        yield start
        start -= 1

# Ambos funcionan igual:
for num in Countdown(5): ...
for num in countdown(5): ...
```

**Cuándo usar clase vs generator**:
- Clase: Necesitas métodos adicionales, estado complejo, reutilización
- Generator: Iteración simple, código más conciso

**Referencia oficial:** [Iterator Types](https://docs.python.org/3/library/stdtypes.html#iterator-types)

In [ ]:
# Example: Range-like class
class CustomRange:
    """
    Custom implementation of range().
    
    :param start: Starting value
    :type start: int
    :param stop: Stopping value (exclusive)
    :type stop: int
    :param step: Step size
    :type step: int
    """
    def __init__(self, start: int, stop: int, step: int = 1) -> None:
        self.start = start
        self.stop = stop
        self.step = step
    
    def __iter__(self) -> 'CustomRange':
        self.current = self.start
        return self
    
    def __next__(self) -> int:
        if self.current >= self.stop:
            raise StopIteration
        value = self.current
        self.current += self.step
        return value
    
    def __repr__(self) -> str:
        return f"CustomRange({self.start}, {self.stop}, {self.step})"

# Works like range()
for i in CustomRange(0, 10, 2):
    print(i)  # 0, 2, 4, 6, 8

## 6. Otros Métodos Mágicos Útiles

### __bool__: Valor de Verdad

Define cómo tu objeto se evalúa en contextos booleanos (if, while, etc.).

In [ ]:
class ShoppingCart:
    """
    Represent a shopping cart.
    """
    def __init__(self) -> None:
        self.items = []
    
    def add_item(self, item: str) -> None:
        """Add item to cart."""
        self.items.append(item)
    
    def __bool__(self) -> bool:
        """Cart is truthy if it has items."""
        return len(self.items) > 0
    
    def __len__(self) -> int:
        return len(self.items)

cart = ShoppingCart()
if cart:
    print("Cart has items")
else:
    print("Cart is empty")  # This prints

cart.add_item("Apple")
if cart:
    print("Cart has items")  # This prints now

### __call__: Objetos Llamables

Permite que instancias de tu clase sean llamadas como funciones.

In [ ]:
class Multiplier:
    """
    Callable object that multiplies by a factor.
    
    :param factor: Multiplication factor
    :type factor: float
    """
    def __init__(self, factor: float) -> None:
        self.factor = factor
    
    def __call__(self, value: float) -> float:
        """Multiply value by factor."""
        return value * self.factor

double = Multiplier(2)
triple = Multiplier(3)

print(double(5))  # 10 - Called like a function!
print(triple(5))  # 15

# Useful for creating function-like objects with state
numbers = [1, 2, 3, 4, 5]
print(list(map(double, numbers)))  # [2, 4, 6, 8, 10]

### __contains__: Operador 'in'

Define el comportamiento del operador `in` para tu clase.

In [ ]:
class Team:
    """
    Represent a team of members.
    
    :param name: Team name
    :type name: str
    """
    def __init__(self, name: str) -> None:
        self.name = name
        self.members = set()
    
    def add_member(self, member: str) -> None:
        """Add member to team."""
        self.members.add(member)
    
    def __contains__(self, member: str) -> bool:
        """Check if member is in team."""
        return member in self.members

team = Team("Python Developers")
team.add_member("Alice")
team.add_member("Bob")

print("Alice" in team)  # True
print("Charlie" in team)  # False

### Aprendizaje Clave

Hay muchos más métodos mágicos: `__add__` para `+`, `__hash__` para usar objetos en sets/dicts, `__enter__`/`__exit__` para context managers. Implementa solo los que necesites para hacer tu clase intuitiva.

**Referencia oficial:** [Special Method Names](https://docs.python.org/3/reference/datamodel.html#special-method-names)

### Respuesta a la Pregunta Anterior

**¿Qué métodos para comportarse como lista?** Mínimo: `__len__` y `__getitem__`. Para funcionalidad completa: también `__setitem__`, `__delitem__`, `__iter__`, `__contains__`, `append()`, `insert()`, etc. Considera heredar de `collections.abc.MutableSequence` para obtener implementaciones por defecto.

## Ejercicios Prácticos

### Tarea 1: Representación de Objetos

Completa los ejercicios en `day_2/exercises/06_magic_methods.py`:

1. Implementa `__str__` y `__repr__` para una clase `Book`
2. Asegúrate de que `__repr__` sea evaluable (pueda recrear el objeto)

### Tarea 2: Comparación de Objetos

1. Implementa `__eq__` y `__lt__` para una clase `Student`
2. Usa `@total_ordering` para obtener todos los operadores de comparación
3. Haz que los estudiantes se ordenen por calificación

### Tarea 3: Contenedores Personalizados

1. Crea una clase `Library` que se comporte como un contenedor
2. Implementa `__len__`, `__getitem__`, y `__contains__`
3. Permite iterar sobre los libros en la biblioteca

### Tarea 4: Iteradores Personalizados

1. Crea un iterador `EvenNumbers` que genere números pares
2. Implementa `__iter__` y `__next__`
3. Asegúrate de que lance `StopIteration` apropiadamente

Ejecuta los tests con:
```bash
pytest day_2/exercises/tests/test_06_magic_methods.py
```

## Resumen

En este notebook has aprendido:

1. **Métodos mágicos** permiten que tus clases se integren con las operaciones integradas de Python
2. **`__str__` y `__repr__`** controlan cómo se representan tus objetos como strings
3. **Métodos de comparación** (`__eq__`, `__lt__`, etc.) permiten comparar y ordenar objetos
4. **`__len__` y `__getitem__`** hacen que tus objetos se comporten como contenedores
5. **`__iter__` y `__next__`** implementan el protocolo de iteración para loops personalizados

### Próximos Pasos

Los métodos mágicos son fundamentales para crear clases pythónicas. En los siguientes notebooks, veremos cómo combinar estos conceptos con otros patrones de diseño para crear código más elegante y mantenible.

## Preguntas de Autoevaluación

### 1. ¿Cuál es la diferencia entre `__str__` y `__repr__`?

**Respuesta:** `__str__` proporciona una representación "amigable" para usuarios finales (legible), mientras que `__repr__` proporciona una representación "técnica" para desarrolladores (sin ambigüedad, idealmente evaluable). `print()` usa `__str__`, mientras que el REPL y `repr()` usan `__repr__`.

### 2. ¿Qué debe retornar un método de comparación cuando se compara con un tipo incompatible?

**Respuesta:** Debe retornar `NotImplemented` (no `False` o lanzar excepción). Esto permite que Python intente la comparación inversa o use el método del otro objeto.

### 3. ¿Qué hace el decorador `@functools.total_ordering`?

**Respuesta:** Genera automáticamente los métodos de comparación faltantes (`__le__`, `__gt__`, `__ge__`) si defines `__eq__` y uno de `__lt__`, `__le__`, `__gt__`, o `__ge__`. Reduce el boilerplate code.

### 4. ¿Cuál es la diferencia entre implementar `__getitem__` y `__iter__`?

**Respuesta:** `__getitem__` permite indexación con `[]` y hace el objeto iterable automáticamente (Python itera usando índices 0, 1, 2...). `__iter__` proporciona control más fino sobre la iteración y es más eficiente para objetos que no son secuencias.

### 5. ¿Cuándo debe un método `__next__` lanzar `StopIteration`?

**Respuesta:** Cuando no hay más elementos para iterar. Esta excepción señala a Python que la iteración ha terminado y es parte del protocolo de iteración.

### 6. ¿Qué método mágico permite usar el operador `in` con tu clase?

**Respuesta:** `__contains__`. Debe retornar `True` si el elemento está en el contenedor, `False` en caso contrario. Si no está implementado, Python intentará iterar sobre el objeto usando `__iter__` o `__getitem__`.

### 7. ¿Para qué sirve el método `__call__`?

**Respuesta:** Permite que instancias de tu clase sean llamadas como funciones. Es útil para crear objetos con estado que se comportan como funciones, como closures o functors.

Discute tus respuestas con compañeros para profundizar tu comprensión.

## Recursos y Referencias Oficiales

### Documentación Oficial

- **Data Model - Python Documentation**: [https://docs.python.org/3/reference/datamodel.html](https://docs.python.org/3/reference/datamodel.html)
  - Documentación completa del modelo de datos de Python
  - Incluye todos los métodos mágicos disponibles

- **Special Method Names**: [https://docs.python.org/3/reference/datamodel.html#special-method-names](https://docs.python.org/3/reference/datamodel.html#special-method-names)
  - Lista completa de métodos especiales
  - Explicación detallada de cada uno

- **Iterator Types**: [https://docs.python.org/3/library/stdtypes.html#iterator-types](https://docs.python.org/3/library/stdtypes.html#iterator-types)
  - Documentación sobre el protocolo de iteración

- **Emulating Container Types**: [https://docs.python.org/3/reference/datamodel.html#emulating-container-types](https://docs.python.org/3/reference/datamodel.html#emulating-container-types)
  - Guía para crear contenedores personalizados

### Módulos Relacionados

- **functools.total_ordering**: [https://docs.python.org/3/library/functools.html#functools.total_ordering](https://docs.python.org/3/library/functools.html#functools.total_ordering)
  - Decorador para generar métodos de comparación

- **collections.abc**: [https://docs.python.org/3/library/collections.abc.html](https://docs.python.org/3/library/collections.abc.html)
  - Abstract Base Classes para contenedores
  - Proporciona implementaciones por defecto de métodos

### Mejores Prácticas

- **Real Python - Python Magic Methods**: [https://realpython.com/python-magic-methods/](https://realpython.com/python-magic-methods/)
  - Tutorial completo con ejemplos prácticos

- **Python Morsels - Every Dunder Method**: [https://www.pythonmorsels.com/every-dunder-method/](https://www.pythonmorsels.com/every-dunder-method/)
  - Lista exhaustiva de todos los métodos mágicos

- **Python Guide - Object-Oriented Programming**: [https://docs.python-guide.org/writing/structure/#object-oriented-programming](https://docs.python-guide.org/writing/structure/#object-oriented-programming)
  - Guía de estilo y mejores prácticas

### Artículos Avanzados

- **Descriptor HowTo Guide**: [https://docs.python.org/3/howto/descriptor.html](https://docs.python.org/3/howto/descriptor.html)
  - Para métodos mágicos avanzados como `__get__`, `__set__`, `__delete__`

- **Python's Data Model**: [https://docs.python.org/3/reference/datamodel.html](https://docs.python.org/3/reference/datamodel.html)
  - Referencia completa del modelo de datos

### Notas Importantes

- Todos los enlaces están actualizados a partir de 2024
- Se recomienda revisar la documentación oficial regularmente
- El Data Model es la fuente más confiable para entender métodos mágicos